# Getting Started with Amazon Nova API

In this notebook, we will walk you through basic set up and understanding the Amazon Nova API as well as introduce you to two new features of the Amazon Nova 2 models.

If you haven't already, we recommend creating a virtual environment as outlined in the [README](/README.md). You can run the cell below to ensure you have the necessary dependencies installed.

In [ ]:
%pip install -r requirements.txt

Through the Amazon Nova API, you will have access to the latest Amazon Nova Models as well as any agents you've created on the Agent Workbench. The code below will help you list all of them.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

#Set up your environment variables
api_key = os.getenv("NOVA_API_KEY")
base_url = "https://api.nova.amazon.com/v1"

# Create OpenAI client
client = OpenAI(api_key=api_key, base_url=base_url)

#List all available models
models = client.models.list()

for model in models.data:
    if model.owned_by == "amazon": # This will only show amazon owned models. Remove this to see your own agents and community agents.
        print(f"{model.display_name}, {model.owned_by}, {model.modelId}")    

For the rest of this notebook, we will use the latest Amazon Nova 2 Lite model, which is a fast, cost-effective reasoning model for most everyday tasks. You can replace this with any model that supports text generation. 

In [ ]:
model_id = "nova-2-lite-v1"  # Replace with your desired model ID

## Non-Streaming Response
Make the first single-turn call to say hello

In [ ]:
## Make the first single turn call to say hello 
response = client.chat.completions.create(
    model=model_id,
    messages=[{
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Hello"
        }
    ]
)

print(response.choices[0].message.content)

## Streaming Responses
Amazon Nova API supports streaming responses for low-latency applications. To enable streaming, set the `stream` parameter to `True` in your request. You can then iterate over the response chunks as they arrive.

We will also use the `max_tokens` parameter to ensure the model has enough room to be verbose and demonstrate the streaming.

In [ ]:
## Make the first single turn call to say hello 
stream = client.chat.completions.create(
    model=model_id,
    messages=[{
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": "Write me a 200 word poem about the ocean."
        },
        
    ],
    stream=True,
    max_tokens=500
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

## Built-in Tools [NEW]
Many Amazon Nova models come with built-in tools to make it easier to develop. At present, these are:
- nova_code_interpreter: useful for running python code in a sanbox environment
- nova_grounding: useful for grounding your responses with web data
By defualt these tools are not enabled, but can be enabled using the `extra_body` parameter.

In [ ]:
# Using Amazon Nova Grounding Tool
response = client.chat.completions.create(
    model=model_id,
    messages=[{
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": "What are the latest developments in renewable energy technology?"
        }],
        extra_body={ 
            "system_tools" : ["nova_grounding"]
        }
)

print(response.choices[0].message.content)

In [ ]:
# Using Amazon Nova Code Interpreter Tool
response = client.chat.completions.create(
    model=model_id,
    messages=[{
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": "Solve for X: X^2 - 4X + 4 = 0"
        }],
        extra_body={ 
            "system_tools" : ["nova_code_interpreter"]
        }
)
print(response.choices[0].message.content)

## Extended Thinking [NEW]
Nova 2 Lite supports extended thinking, improving accuracy for tasks requiring multi-step reasoning, mathematical problem-solving, and logical analysis. 

The Nova API supports this via the `reasoning_effort` parameter with options `low`, `medium`, and `high`.   

When using `high` the model will default max_tokens to 64,000. When using other modes, we recommend at least 10,000 tokens.

In [ ]:
## Make the first single turn call to say hello 
response = client.chat.completions.create(
    model=model_id,
    messages=[{
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": """I need to optimize a logistics network with 5 warehouses, 12 distribution centers, and 200 retail locations. 
            The goal is to minimize total transportation costs while ensuring no location is more than 50 miles from a distribution center. 
            What approach should I take?"""
        },
    ],
    reasoning_effort="medium",
    max_tokens=40000
)

print(response.choices[0].message.content)